In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
import importlib
import performance_shr
importlib.reload(performance_shr)
from performance_shr import convert_to_float

In [3]:
default_threshold = 0.10   # eg 0.1 = 10%
file_dir = 'D:/Astro/Images/2024-12-23/Lump Star/LIGHTS'
file_name = 'ImageMetaData.csv'
file_path = os.path.join(file_dir, file_name)
converters_dict = {
    'FilePath': str,   
    'Gain': convert_to_float,   
    'DetectedStars': convert_to_float,   
    'HFR': convert_to_float,  
    'HFRStDev': convert_to_float,  
    'FWHM': convert_to_float,  
    'Eccentricity': convert_to_float,  
    'FocuserPosition': convert_to_float,  
    'Airmass': convert_to_float,  
    'MountRA': convert_to_float,  
    'MountDec': convert_to_float,  
    }

In [4]:
data = pd.read_csv(file_path, converters=converters_dict)
data['FileName'] = data['FilePath'].apply(lambda x: os.path.basename(x))
data = data[data['FilePath'].str.strip() != '' ]
data

,ExposureNumber,FilePath,FilterName,ExposureStart,Duration,Binning,CameraTemp,CameraTargetTemp,Gain,Offset,...,GuidingRMSDECArcSec,FocuserPosition,FocuserTemp,RotatorPosition,PierSide,Airmass,ExposureStartUTC,MountRA,MountDec,FileName
0,0,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-23 22:06,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.4291,2024-12-23T11:06:02.3260561Z,85.394743,-2.306097,2024-12-23_22-06-02_NoFilter_400_10.00s_0000.cr3
1,1,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-23 22:06,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.4282,2024-12-23T11:06:14.0857099Z,85.395147,-2.306352,2024-12-23_22-06-14_NoFilter_400_10.00s_0001.cr3
2,2,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-23 22:06,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.4273,2024-12-23T11:06:26.0663062Z,85.395101,-2.306175,2024-12-23_22-06-26_NoFilter_400_10.00s_0002.cr3
3,3,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-23 22:06,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.4265,2024-12-23T11:06:37.9860814Z,85.395425,-2.306343,2024-12-23_22-06-37_NoFilter_400_10.00s_0003.cr3
4,4,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-23 22:06,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.4256,2024-12-23T11:06:49.9818161Z,85.395451,-2.306301,2024-12-23_22-06-49_NoFilter_400_10.00s_0004.cr3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,1565,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-24 03:28,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.6470,2024-12-23T16:28:58.3656325Z,85.378464,-2.295301,2024-12-24_03-28-58_NoFilter_400_10.00s_1565.cr3
1566,1566,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-24 03:29,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.6484,2024-12-23T16:29:10.1338425Z,85.378713,-2.295177,2024-12-24_03-29-10_NoFilter_400_10.00s_1566.cr3
1567,1567,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-24 03:29,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.6498,2024-12-23T16:29:21.9461777Z,85.379177,-2.294949,2024-12-24_03-29-21_NoFilter_400_10.00s_1567.cr3
1568,1568,D:/Astro/Images/2024-12-23/Lump Star/LIGHTS/20...,NoFilter,2024-12-24 03:29,10,1x1,NaN,NaN,400.0,-1,...,0,200.0,NaN,NaN,NaN,1.6512,2024-12-23T16:29:33.7924868Z,85.379338,-2.295061,2024-12-24_03-29-33_NoFilter_400_10.00s_1568.cr3


In [47]:
def filter_percentiles(df, col='HFR', top=default_threshold, bottom=default_threshold, zero=True):
    top_threshold = df[col].quantile(1 - top) if top else False
    bottom_threshold = df[col].quantile(bottom) if bottom else False
    is_na = df[col].isna()
    is_false = is_na != is_na
    is_zero = df[col] == 0 if zero else is_false
    is_top = df[col] >= top_threshold if top_threshold else is_false
    is_bottom = df[col] <= bottom_threshold if bottom_threshold else is_false
    is_filtered = is_zero | is_na | is_top | is_bottom

    fig=go.Figure()
    color_map = {True: 'red', False: 'blue'}
    fig.add_trace(go.Scatter(x=df.ExposureNumber, y=df[col], name=col, mode='markers', marker=dict(color=np.vectorize(color_map.get)(is_filtered))))
    fig.update_layout(title=f'{col} vs Exposure No.', xaxis_title='Exposure No.', yaxis_title=col)
    fig.show()    

    print(f'{col}: TopThreshold={top_threshold:.3f} n({is_top.sum()}), BottomThreshold={bottom_threshold:.3f} n({is_bottom.sum()}), na=n({is_na.sum()}), zero=n({is_zero.sum()}), filtered=n({is_filtered.sum()})\n\n')
    return is_filtered

In [48]:
print(f'**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****')
is_badHFR = filter_percentiles(data, col='HFR', top=0.10, bottom=False)
is_badHFRStDev = filter_percentiles(data, col='HFRStDev', top=0.05, bottom=0.01)
is_badFWHM = filter_percentiles(data, col='FWHM', top=0.08, bottom=0.01)
is_badEccentricity = filter_percentiles(data, col='Eccentricity', top=0.10, bottom=0.10)
is_badDetectedStars = filter_percentiles(data, col='DetectedStars', top=False, bottom=0.10)
is_rejected = is_badFWHM | is_badHFR | is_badFWHM | is_badHFRStDev | is_badEccentricity | is_badDetectedStars
print(f'\nTotal Rejected: n({is_rejected.sum()}/{len(data)})')


**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****


HFR: TopThreshold=2.669 n(157), BottomThreshold=0.000 n(0), na=n(0), zero=n(0), filtered=n(157)




HFRStDev: TopThreshold=0.293 n(79), BottomThreshold=0.156 n(16), na=n(0), zero=n(0), filtered=n(95)




FWHM: TopThreshold=11.176 n(124), BottomThreshold=8.083 n(16), na=n(21), zero=n(0), filtered=n(161)




Eccentricity: TopThreshold=0.628 n(155), BottomThreshold=0.331 n(155), na=n(21), zero=n(1), filtered=n(331)




DetectedStars: TopThreshold=0.000 n(0), BottomThreshold=590.900 n(157), na=n(0), zero=n(0), filtered=n(157)



Total Rejected: n(480/1570)


In [22]:
# Create the 'rejected' directory at the same level as file_dir if it doesn't exist 
parent_dir = os.path.dirname(file_dir) 
rejected_dir = os.path.join(parent_dir, 'Rejected') 
os.makedirs(rejected_dir, exist_ok=True)

In [24]:
rejects = data[is_rejected].FileName
for filename in rejects:
    src_path = os.path.join(file_dir, filename)
    dest_path = os.path.join(rejected_dir, filename)
    if os.path.exists(src_path): 
        shutil.move(src_path, dest_path) 
        print(f'Moved: {filename}')

print('File moving complete.')

Moved: 2024-12-20_22-17-22_NoFilter_400_15.00s_0000.cr3
Moved: 2024-12-20_22-20-57_NoFilter_400_15.00s_0012.cr3
Moved: 2024-12-20_22-22-09_NoFilter_400_15.00s_0016.cr3
Moved: 2024-12-20_22-24-32_NoFilter_400_15.00s_0024.cr3
Moved: 2024-12-20_22-24-50_NoFilter_400_15.00s_0025.cr3
Moved: 2024-12-20_22-25-08_NoFilter_400_15.00s_0026.cr3
Moved: 2024-12-20_22-25-26_NoFilter_400_15.00s_0027.cr3
Moved: 2024-12-20_22-26-02_NoFilter_400_15.00s_0029.cr3
Moved: 2024-12-20_22-26-37_NoFilter_400_15.00s_0031.cr3
Moved: 2024-12-20_22-31-05_NoFilter_400_15.00s_0046.cr3
Moved: 2024-12-20_22-31-23_NoFilter_400_15.00s_0047.cr3
Moved: 2024-12-20_22-31-41_NoFilter_400_15.00s_0048.cr3
Moved: 2024-12-20_22-31-59_NoFilter_400_15.00s_0049.cr3
Moved: 2024-12-20_22-32-35_NoFilter_400_15.00s_0051.cr3
Moved: 2024-12-20_22-33-10_NoFilter_400_15.00s_0053.cr3
Moved: 2024-12-20_22-33-28_NoFilter_400_15.00s_0054.cr3
Moved: 2024-12-20_22-33-46_NoFilter_400_15.00s_0055.cr3
Moved: 2024-12-20_22-34-22_NoFilter_400_15.00s_0